Notebook Training Run on Kaggle:
https://www.kaggle.com/code/ahmedkhalifa1999/oxford-iiit-pet-classification-resnet50

In [ ]:
import torch

In [ ]:
from torchvision.datasets import OxfordIIITPet
from torchvision.transforms import ToTensor, Compose, Resize, Normalize, RandomResizedCrop, RandomHorizontalFlip, RandomRotation, ColorJitter
from torch.utils.data import random_split, DataLoader

batch_size = 128
image_size = 224

train_data_transform = Compose([
    Resize((image_size, image_size)),
    RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio = (0.9, 1.1)),
    RandomHorizontalFlip(),
    RandomRotation(7),
    ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.25),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_transform = Compose([
    Resize((image_size, image_size)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trainval_dataset = OxfordIIITPet(root = '../data', split = 'trainval', transform = train_data_transform, target_types = 'category', download = True)

train_dataset, val_dataset = random_split(dataset = trainval_dataset, lengths = [0.8, 0.2])

test_dataset = OxfordIIITPet(root = '../data', split = 'test', transform = data_transform, target_types = 'category', download = True)

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, pin_memory = True)
val_dataloader = DataLoader(val_dataset, batch_size, pin_memory = True)
test_dataloader = DataLoader(test_dataset, batch_size, pin_memory = True)

In [ ]:
print(f'Training Dataset Length: {len(train_dataset)}')
print(f'Validation Dataset Length: {len(val_dataset)}')
print(f'Testing Dataset Length: {len(test_dataset)}')

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(ResNet50_Weights.DEFAULT)

In [ ]:
from torch import nn

model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 37)
)

In [ ]:
device = torch.device("cuda")

model.to(device);

In [ ]:
from torch import optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 1e-3)

In [ ]:
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0.0
    batch_count = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()                
        optimizer.step()

        print(f'[{epoch + 1}, {i + 1}] loss: {loss}')

        epoch_loss += loss.item()
        batch_count += 1

    # Calculate Accuracy on training and validation datasets
    correct = 0
    total = 0
    with torch.no_grad():
        for data in train_dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_accuracy = correct / total

    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
    
    print(f'Epoch {epoch + 1} loss: {epoch_loss/batch_count}, train accuracy: {train_accuracy}, val accuracy: {val_accuracy}')

In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_accuracy = correct / total

print(f'test accuracy: {test_accuracy}')

In [ ]:
torch.save(model.state_dict(), '../models/PetClassification-Resnet50.pth')